In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters for the simulation (aligned with observed trends in fatigue data)
tau_f_prime = 1390  # Shear fatigue strength coefficient (τ'_f) in MPa
gamma_f_prime = 0.40  # Shear fatigue ductility coefficient (γ'_f)
b_0 = -0.055  # Adjusted Shear fatigue strength exponent (for decaying trend)
c_0 = -0.45  # Adjusted Shear fatigue ductility exponent (for decaying trend)
G = 45.7e3  # Shear modulus in MPa
sigma_n_max = 200  # Maximum normal stress in MPa
sigma_y = 526  # Yield strength in MPa
k = 1.5  # Material-dependent parameter for FS

# Generate simulated reversals to failure (log-distribution to simulate realistic fatigue life data)
Nf = np.logspace(3, 7, num=100)  # Simulating between 10^3 and 10^7 reversals to failure

# Simulating defect distribution (random voids, etc.)
defect_factor = np.random.uniform(0.95, 1.05, size=Nf.shape)  # Random defect multiplier (small variation)

# FS parameter calculation function (elastic + plastic strain components)
def fs_parameter_simulation(N, tau_f_prime, G, gamma_f_prime, b_0, c_0, sigma_n_max, sigma_y, k, defect_factor):
    elastic_term = (tau_f_prime / G) * (2 * N) ** b_0
    plastic_term = gamma_f_prime * (2 * N) ** c_0
    fs_value = (elastic_term + plastic_term) * (1 + k * sigma_n_max / sigma_y) * defect_factor
    return fs_value

# Calculate the simulated FS parameters
fs_parameters_simulated = fs_parameter_simulation(Nf, tau_f_prime, G, gamma_f_prime, b_0, c_0, sigma_n_max, sigma_y, k, defect_factor)

# Plot the simulated FS parameters vs. reversals to failure (2N_f)
plt.figure(figsize=(8, 6))
plt.loglog(2 * Nf, fs_parameters_simulated, 'bo', alpha=0.5, label='Simulated FS Data')

# Add labels and title
plt.title('Simulated FS Damage Parameter vs. Reversals to Failure', fontsize=14)
plt.xlabel('Reversals to Failure (2N_f)', fontsize=12)
plt.ylabel('FS Damage Parameter', fontsize=12)
plt.grid(True, which="both", ls="--", lw=0.5)
plt.legend()
plt.tight_layout()

# Show plot
plt.show()
